# Load data in

In [ ]:
import numpy as np
from pathlib import Path

def reshape_array(arr):
    return arr.reshape(arr.shape[0], arr.shape[1], 1)

def load_data_in(folder_name, prefix):
    current_dir = Path('./')
    data_dir = current_dir / folder_name
    train_data = data_dir / '{}_train_reverb.npy'.format(prefix)
    train_label = current_dir / 'y_train.npy'
    test_data = data_dir / '{}_test_reverb.npy'.format(prefix)
    test_label = current_dir / 'y_test.npy'
    
    with open(train_data, 'rb') as f:
        X_train_data = np.load(f)
        X_train_data = X_train_data.reshape(X_train_data.shape[0], X_train_data.shape[1], X_train_data.shape[2], 1)
    with open(test_data, 'rb') as f:
        X_test_data = np.load(test_data)
        X_test_data = X_test_data.reshape(X_test_data.shape[0], X_test_data.shape[1], X_test_data.shape[2], 1)

    with open(train_label, 'rb') as f:
        y_train = np.load(f)
        
    with open(test_label, 'rb') as f:
        y_test = np.load(f)
        
    return X_train_data, y_train, X_test_data, y_test

# CNN model

In [ ]:

import tensorflow as tf
from sklearn.metrics import classification_report
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 1GB of memory on the first GPU
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5*1024)])
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
        print(e)
        
print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")

def generate_cnn_model(X_im):

    n_features = X_im.shape[1]
    time_steps = X_im.shape[2]
    print(n_features, time_steps)

    model_cnn = tf.keras.models.Sequential()
    model_cnn.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(n_features, time_steps, 1))) # convolve with 32 kernels of size 3 x 3
    model_cnn.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same')) # convolve with 32 kernels of size 3 x 3
    model_cnn.add(tf.keras.layers.MaxPooling2D((2, 2))) #Dowmsample by 2 in each direction- take max element of every 2
    model_cnn.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same')) # convolve with 64 kernels of size 3 x 3
    model_cnn.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model_cnn.add(tf.keras.layers.MaxPooling2D((2, 2)))
    model_cnn.add(tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same')) # convolve with 128 kernels of size 3 x 3
    model_cnn.add(tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model_cnn.add(tf.keras.layers.MaxPooling2D((2, 2)))
    model_cnn.add(tf.keras.layers.Flatten()) # Flatten output into a vector
    model_cnn.add(tf.keras.layers.Dense(128, activation='relu', kernel_initializer='he_uniform')) # Fully connected layer with 128 nodes
    model_cnn.add(tf.keras.layers.Dense(10, activation='softmax')) #output layer, size must equal the number of classes

    Adam=tf.keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.9, amsgrad=False)

    model_cnn.compile(loss='categorical_crossentropy',
                  optimizer=Adam,
                  metrics=['accuracy'])
    model_cnn.summary() #print out breakdown of model parameters
    
    return model_cnn

def evaluate_approach(folder_name, prefix):
    X_train, y_train, X_test, y_test = load_data_in(folder_name, prefix)
    print(X_test.shape, y_test.shape)
    print(X_train.shape, y_train.shape)

    model = generate_cnn_model(X_train)
    
    hist = model.fit(x=X_train, y=y_train, epochs=500, batch_size=200)
    model.evaluate(X_test, y_test)
    
    y_pred = model.predict(X_test, batch_size=200)
    y_pred = (y_pred > 0.5) 
    
    return model, classification_report(y_test, y_pred)


In [ ]:
model, report = evaluate_approach('FE_data', 'MFCC')
print(report)

In [ ]:
model, report = evaluate_approach('FE_data', 'LPC')
print(report)

In [ ]:
model, report = evaluate_approach('FE_data', 'STFT')
print(report)

In [ ]:
model, report = evaluate_approach('FE_data', 'GFCC')
print(report)

In [ ]:
model, report = evaluate_approach('FE_data', 'PLP')
print(report)